# Closures

In [10]:
struct City {
    name: String,
    population: i64,
    country: String,
}

let mut cities = vec![
    City { 
        name: "Tokyo".to_string(), 
        population: 37_833_000, 
        country: "Japan".to_string() 
    },
    City { 
        name: "Delhi".to_string(), 
        population: 29_399_141, 
        country: "India".to_string() 
    },
    City { 
        name: "Shanghai".to_string(), 
        population: 26_317_104, 
        country: "China".to_string() 
    },
    City { 
        name: "São Paulo".to_string(), 
        population: 21_650_000, 
        country: "Brazil".to_string() 
    },
    City {
        name: "Cracow".to_string(),
        population: 769498,
        country: "Poland".to_string()
    }
];

cities.sort_by_key(|city| -city.population);

println!("Cities ordered by the population (descending):");
println!("-----------------------------------");
for city in &cities {
    println!("{}, {} - population: {}", city.name, city.country, city.population);
}


Cities ordered by the population (descending):
-----------------------------------
Tokyo, Japan - population: 37833000
Delhi, India - population: 29399141
Shanghai, China - population: 26317104
São Paulo, Brazil - population: 21650000
Cracow, Poland - population: 769498


()

## Closures that borrow

In [3]:
use std::collections::HashMap;

let mut cities_area = HashMap::new();

cities_area.insert("Tokyo".to_string(), 2187);
cities_area.insert("Delhi".to_string(), 1484);
cities_area.insert("Shanghai".to_string(), 6340);
cities_area.insert("São Paulo".to_string(), 1521);
cities_area.insert("Cracow".to_string(), 326);

In [4]:
cities.sort_by_key(|city| -cities_area[&city.name]);

In [5]:
println!("\nCities ordered by the area (descending):");
println!("-----------------------------------");
for city in &cities {
    println!("{}, {} - area: {} km²", city.name, city.country, cities_area[&city.name]);
}


Cities ordered by the area (descending):
-----------------------------------
Shanghai, China - area: 6340 km²
Tokyo, Japan - area: 2187 km²
São Paulo, Brazil - area: 1521 km²
Delhi, India - area: 1484 km²
Cracow, Poland - area: 326 km²


()

## Closures that move

In [6]:
use std::thread;

fn start_sorting_thread(mut cities: Vec<City>, cities_area: HashMap<String, i32>) -> thread::JoinHandle<Vec<City>> {
    let key_fn = move |city: &City| -> i32 { -cities_area[&city.name] };

    thread::spawn(move || {
        cities.sort_by_key(key_fn);
        cities
    })
}

start_sorting_thread(cities, cities_area).join().unwrap();

* Rust offers two ways for closures to get data from enclosing scopes: moves and borrowing
* If a closure would move a value of a copyable type, like `i32`, it copies the value instead
* Values of noncopyable types, like `Vec<City>`, really are moved: the preceding code transfers cities to the new thread, by way of the move closure 

## Function & Closure Types

### Function Type (functions only)

In [17]:
fn is_city_big(city: &City) -> bool {
    city.population > 10_000_000
}

* Function above has a type: `fn(&City) -> i64`

In [18]:
let city_predicate: fn(&City) -> bool = is_city_big;

In [19]:
city_predicate(&cities[0])

true

* We can pass function as an argument to another function.

In [21]:
fn count_selected_cities(cities: &Vec<City>, predicate: fn(&City) -> bool) -> i32 {
    let mut count = 0;
    for city in cities {
        if predicate(city) {
            count += 1;
        }
    }
    count

    // alternative one liner using iterator
    // cities.iter().filter(|city| predicate(city)).count() as i32
}

In [22]:
count_selected_cities(&cities, is_city_big)

4

### Fn Trait - functions & closures

In [24]:
let size_limit = 20_000_000;

let is_big_enough = |city: &City| -> bool { city.population > size_limit };

Error: The variable `is_big_enough` is a closure, which cannot be persisted.
You can however persist closures if you box them. e.g.:
let f: Box<dyn Fn()> = Box::new(|| {println!("foo")});
Alternatively, you can prevent evcxr from attempting to persist
the variable by wrapping your code in braces.

In [30]:
let size_limit = 20_000_000;

let is_big_enough: Box<dyn Fn(&City)->bool> = Box::new(|city: &City| -> bool { city.population > size_limit });

Error: `size_limit` does not live long enough

In [32]:
let size_limit = 20_000_000;

let is_big_enough: Box<dyn Fn(&City)->bool> = Box::new(move |city: &City| -> bool { city.population > size_limit });

In [33]:
count_selected_cities(&cities, is_big_enough)

Error: mismatched types

In [34]:
fn count_selected_cities(cities: &Vec<City>, predicate: impl Fn(&City) -> bool) -> i32 {
    let mut count = 0;
    for city in cities {
        if predicate(city) {
            count += 1;
        }
    }
    count
}

In [36]:
count_selected_cities(&cities, is_big_enough)

4

### FnOnce

In [45]:
let my_string = "Hello, World!".to_string();


Error: cannot move out of `my_string`, a captured variable in an `Fn` closure

In [47]:
fn call_twice<F>(closure: F) 
where F: Fn() {
    closure();
    closure();
}

In [48]:
let my_string = "Hello, World!".to_string();

call_twice(|| println!("{}", my_string));

Hello, World!
Hello, World!


In [49]:
call_twice(|| drop(my_string))

Error: cannot move out of `my_string`, a captured variable in an `Fn` closure

* Closures that drop values are not allowed to have `Fn`. They are, quite literally, no `Fn` at all. They implement a less powerful trait, `FnOnce`, the trait of closures that can be called once.

In [55]:
// mod explain {
//     trait Fn() -> R {
//         fn call(&self) -> R;
//     }

//     trait FnOnce() -> R {
//         fn call_once(self) -> R;
//     }
//     
//     trait FnMut() -> R {
//         fn call_mut(&mut self) -> R;
//     }
// }

* `Fn` is the family of closures and functions that you can call multiple times without restriction. This highest category also includes all `fn` functions.
* `FnMut` is the family of closures that can be called multiple times if the closure itself is declared mut.
* `FnOnce` is the family of closures that can be called once, if the caller owns the closure.

* `Fn()` is a subtrait of `FnMut()`, which is a subtrait of `FnOnce()`. 
* This makes `Fn` the most exclusive and most powerful category. `FnMut` and `FnOnce` are broader categories that include closures with usage restrictions.

## Copy and Clone for Closures

* Just as Rust automatically figures out which closures can be called only once, it can figure out which closures can implement `Copy` and `Clone`, and which cannot.

In [56]:
let y = 4;
let add_y = |x| x + y;
let copy_of_add_y = add_y; // closure is copied
assert_eq!(add_y(copy_of_add_y(3), 11);

Error: The variable `add_y` is a closure, which cannot be persisted.
You can however persist closures if you box them. e.g.:
let f: Box<dyn Fn()> = Box::new(|| {println!("foo")});
Alternatively, you can prevent evcxr from attempting to persist
the variable by wrapping your code in braces.